# Dazbo's YouTube and Video Demos

## Overview

Examples of how to work with YouTube videos using Python. Here I'll demonstrate:

- How to [download videos and extract audio](#downloading-videos-and-extracting-audio)
- How to [transcribe audio to text using a speech-to-text API](#extracting-audio-using-python-speech-recognition)
- How to [extract existing transcripts and translate](#extract-existing-transcripts-from-videos)

**To run this notebook, first execute the cells in the [Setup](#Setup) section, as described below.** Then you can experiment with any of the subsequent cells.

A few useful notes:

- The source for this notebook source lives in my GitHub repo, <a href="https://github.com/derailed-dash/youtube-and-video" target="_blank">Youtube-and-Video</a>.
- Check out further guidance - including tips on how to run the notebook, in the project's `README.md`.
- For example, you could...
  - Run the notebook locally, in your own Jupyter environment.
  - Run the notebook in a cloud-based Jupyter environment, with no setup required on your part! For example, with **Google Colab**: <br><br><a href="https://colab.research.google.com/github/derailed-dash/youtube-and-video/blob/main/src/notebooks/youtube-demos.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Google Colab"/></a><br><br>It looks like this:<br><br><img src="https://github.com/derailed-dash/youtube-and-video/blob/main/src/notebooks/static/images/collab-view.png?raw=1" width="640px"></img>
- For more ways to run Jupyter Notebooks, check out [my guide](https://medium.com/python-in-plain-english/five-ways-to-run-jupyter-labs-and-notebooks-23209f71e5c0).


## Setup

### Packages

First, let's install any dependent packages:

In [1]:
%pip install --upgrade --no-cache-dir python-dotenv dazbo-commons pytubefix moviepy yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.6 MB/s eta 0:00:00


In [2]:
import IPython
from IPython.display import display
from IPython.core.display import Markdown

import logging
import re
import io
import sys
from pathlib import Path
from dataclasses import dataclass
import dazbo_commons as dc
from dotenv import load_dotenv

In [3]:
# Colab requires an older version of Ipykernel
if not "google.colab" in sys.modules:
    pass
    %pip install --upgrade --no-cache-dir ipykernel


### Logging

Now we'll setup logging. Here I'm using coloured logging from my [dazbo-commons](https://pypi.org/project/dazbo-commons/) package. Feel free to change the logging level.

In [4]:
# Setup logging
APP_NAME="dazbo-yt-demos"
logger = dc.retrieve_console_logger(APP_NAME)
logger.setLevel(logging.DEBUG)
logger.info("Logger initialised.")
logger.debug("DEBUG level logging enabled.")

14:54:43.806:dazbo-yt-demos - INF: Logger initialised.
14:54:43.809:dazbo-yt-demos - DBG: DEBUG level logging enabled.


### File Locations

Here we initialise some file path locations, e.g. an output folder.

In [5]:
locations = dc.get_locations(APP_NAME)
for attribute, value in vars(locations).items():
    logger.debug(f"{attribute}: {value}")

14:54:43.823:dazbo-yt-demos - DBG: script_name: dazbo-yt-demos
14:54:43.826:dazbo-yt-demos - DBG: script_dir: /content/dazbo-yt-demos
14:54:43.828:dazbo-yt-demos - DBG: input_dir: /content/dazbo-yt-demos/input
14:54:43.830:dazbo-yt-demos - DBG: output_dir: /content/dazbo-yt-demos/output
14:54:43.835:dazbo-yt-demos - DBG: input_file: /content/dazbo-yt-demos/input/input.txt


### Utility Functions

In [6]:
def clean_filename(filename):
    """ Create a clean filename by removing unallowed characters. """
    pattern = r'[^a-zA-Z0-9._\s-]'
    return  re.sub(pattern, '_', filename)

### Install Additional Packages

- [ffmpeg](https://ffmpeg.org/): a useful utility for video and audio format conversion. Many Python libraries use it. It will not generally be used by this notebook, but if you run into errors requiring ffmpeg, you will want to run this section.

In [7]:
import os
import platform
import subprocess

def run_command(command):
    """Run a shell command and print its output in real-time."""
    process = subprocess.Popen(
        command,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    # Read and print the output line by line
    if process.stdout is not None:
        for line in iter(process.stdout.readline, b''):
            logger.info(line.decode().strip())
        process.stdout.close()

    process.wait()

def install_ffmpeg():
    """ Install required system software, like ffmpeg """
    os_name = platform.system()
    logger.info(f"Installing packages on {os_name}...")

    if os_name == "Windows":
        run_command("winget install ffmpeg --no-upgrade")
    elif os_name == "Linux":
        run_command("apt-get -qq update && apt-get -qq -y install ffmpeg")
    elif os_name == "Darwin":
        run_command("brew install ffmpeg")
    else:
        logger.error(f"Unsupported operating system: {os_name}")

try:
    logger.debug("Checking if ffmpeg is installed")
    output = subprocess.check_output(["ffmpeg", "-version"])
    logger.debug(f"ffmpeg version: {output.decode().strip()}")
    mpeg_installed = True
    logger.debug("ffmpeg is already installed.")
except subprocess.CalledProcessError:
    logger.debug("ffmpeg is not installed.")
    mpeg_installed = False

if not mpeg_installed:
    logger.info("Installing ffmpeg...")
    install_ffmpeg()


14:54:43.859:dazbo-yt-demos - DBG: Checking if ffmpeg is installed
14:54:44.956:dazbo-yt-demos - DBG: ffmpeg version: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --

Now we'll check `ffmpeg` has been installed.

On Windows, this may not have been added to your path. If so, you can check your default install location using `winget --info`, and then add it to your path.

In [8]:
logger.info("Note that installed applications may not be immediately available after first installing.\n" \
            "It may be necessary to relaunch the notebook environment.")

!ffmpeg -version

14:54:44.976:dazbo-yt-demos - INF: Note that installed applications may not be immediately available after first installing.
It may be necessary to relaunch the notebook environment.


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

### Videos to Work With

We start by defining a list of videos to test our application with, along with a function that takes a full YouTube URL and returns just the id portion.

I’ve used these videos because…

- The first is the fantastic [Burning Bridges](https://www.youtube.com/watch?v=udRAIF6MOm8) by Sigrid. The video has no embedded transcript.
- The second is the beautiful song [I Believe](https://www.youtube.com/watch?v=CiTn4j7gVvY) by Melissa Hollick. It’s one of my favourite songs of all time. When I get a migraine, I turn off the lights, and listen to this to feel better! And for those who enjoy gaming, this song is the end titles to the amazing Wolfenstein: New Order game. This video has an embedded transcript.
- Then we have a short [Jim Carey speech](https://www.youtube.com/watch?v=nLgHNu2N3JU), which gives us dialog without music or other ambient noise. It has an embedded transcript.
- And finally, a [Ukrainian song](https://www.youtube.com/watch?v=d4N82wPpdg8) from Eurovision 2024, by Jerry Heil and Alyona Alyona. This gives us an opportunity to test translation. It also has an embedded transcript.

In [9]:
# Videos to download
# urls = [
#     "https://www.youtube.com/watch?v=udRAIF6MOm8",  # Sigrid - Burning Bridges (English)
#     "https://www.youtube.com/watch?v=CiTn4j7gVvY",  # Melissa Hollick - I Believe (English)
#     "https://www.youtube.com/watch?v=nLgHNu2N3JU",  # Jim Carey - Motivational speech (English)
#     "https://www.youtube.com/watch?v=d4N82wPpdg8",  # Jerry Heil & Alyona Alyona - Teresa & Maria (Ukrainian)
# ]

urls = [
    "https://www.youtube.com/watch?v=qcSOav9Opko",  # Alain Connes : « Un topo sur les topos » — Lectures grothendieckiennes.
]

def get_video_id(url: str) -> str:
    """ Return the video ID, which is the part after 'v=' """
    return url.split("v=")[-1]

## Downloading Videos and Extracting Audio

Here I'll demonstrate a few different Python libraries for working with YouTube videos.

### Option 1 - With PyTubeFix

Here I'll use the [pytubefix](https://github.com/JuanBindez/pytubefix) library to download YouTube videos, and then to download mp3 audio-only streams as files.

This library is a community-maintained fork of `pytube`. It was created to provide quick fixes for issues that the official pytube library faced, particularly when YouTube's updates break `pytube`.

Pros:

- The library is very easy to use.
- We can work with video, audio, channels, playlists, and even search and filter.
- It is [well documented](https://pytubefix.readthedocs.io/en/latest/).
- It can be used from the command line, with its simple CLI.
- It is VERY FAST!

Cons:

- Does not offer some of the more sophisticated capabilities that are offered by `yt_dlp`.
- It does not appear to set mp3 headers correctly. The mp3s are actually encoded as mp4a. I don't think this is a problem, but it's worth bearing in mind!

In [23]:

from pytubefix import YouTube
from pytubefix.cli import on_progress

output_locn = f"{locations.output_dir}/pytubefix"

def process_yt_videos():
    for i, url in enumerate(urls):
        logger.info(f"Downloads progress: {i+1}/{len(urls)}")

        try:
            yt = YouTube(url, on_progress_callback=on_progress)
            logger.info(f"Getting: {yt.title}")
            video_stream = yt.streams.get_highest_resolution()
            if not video_stream:
                raise Exception("Stream not available.")

            # YouTube resource titles may contain special characters which
            # can't be used when saving the file. So we need to clean the filename.
            cleaned = clean_filename(yt.title)

            video_output = f"{output_locn}/{cleaned}.mp4"
            logger.info(f"Downloading video {cleaned}.mp4 ...")
            video_stream.download(output_path=output_locn, filename=f"{cleaned}.mp4")

            logger.info(f"Creating audio...")
            audio_stream = yt.streams.get_audio_only()
            audio_stream.download(output_path=output_locn, filename=cleaned, mp3=True)

            logger.info("Done")

        except Exception as e:
            logger.error(f"Error processing URL '{url}'.")
            logger.debug(f"The cause was: {e}")

    logger.info(f"Downloads finished. See files in {output_locn}.")

process_yt_videos()


15:18:17.436:dazbo-yt-demos - INF: Downloads progress: 1/4
15:18:17.534:dazbo-yt-demos - INF: Getting: Sigrid - Burning Bridges
15:18:17.536:dazbo-yt-demos - INF: Downloading video Sigrid - Burning Bridges.mp4 ...
15:18:17.583:dazbo-yt-demos - INF: Creating audio...
15:18:17.585:dazbo-yt-demos - INF: Done
15:18:17.587:dazbo-yt-demos - INF: Downloads progress: 2/4
15:18:17.681:dazbo-yt-demos - INF: Getting: Wolfenstein: The New Order - I Believe - Melissa Hollick (Official Ending Song)
15:18:17.683:dazbo-yt-demos - INF: Downloading video Wolfenstein_ The New Order - I Believe - Melissa Hollick _Official Ending Song_.mp4 ...
15:18:17.686:dazbo-yt-demos - INF: Creating audio...
15:18:17.689:dazbo-yt-demos - INF: Done
15:18:17.691:dazbo-yt-demos - INF: Downloads progress: 3/4
15:18:17.786:dazbo-yt-demos - INF: Getting: Jim Carrey Commencement Speech His Father's Inspiration
15:18:17.788:dazbo-yt-demos - INF: Downloading video Jim Carrey Commencement Speech His Father_s Inspiration.mp4 ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Option 2 - PyTubeFix and MoviePy

Here I'm doing the same as before, but I'm extracting the audio using the Python [MoviePy](https://github.com/Zulko/moviepy) library. This is a powerful video and audio editing library.

Pros:

- We can extract audio as mp3 with correct headers.
- It is [well documented](https://zulko.github.io/moviepy/).
- It is powerful.

Cons:

- It is slower to extract the audio than using `pytubefix` alone.

In [11]:

from pytubefix import YouTube
from pytubefix.cli import on_progress
from moviepy.editor import VideoFileClip

output_locn = f"{locations.output_dir}/pytubefix_with_moviepy"

def process_yt_videos():
    for i, url in enumerate(urls):
        logger.info(f"Downloads progress: {i+1}/{len(urls)}")

        try:
            yt = YouTube(url, on_progress_callback=on_progress)
            logger.info(f"Getting: {yt.title}")
            video_stream = yt.streams.get_highest_resolution()
            if not video_stream:
                raise Exception("Stream not available.")

            # YouTube resource titles may contain special characters which
            # can't be used when saving the file. So we need to clean the filename.
            cleaned = clean_filename(yt.title)

            video_output = f"{output_locn}/{cleaned}.mp4"
            logger.info(f"Downloading video {cleaned}.mp4 ...")
            video_stream.download(output_path=output_locn, filename=f"{cleaned}.mp4")

            logger.info(f"Creating audio...")
            video_clip = VideoFileClip(video_output) # purely to give us access to methods
            assert video_clip.audio is not None
            video_clip.audio.write_audiofile(f"{output_locn}/{cleaned}.mp3")
            video_clip.close()

            logger.info("Done")

        except Exception as e:
            logger.error(f"Error processing URL '{url}'.")
            logger.debug(f"The cause was: {e}")

    logger.info(f"Downloads finished. See files in {output_locn}.")

process_yt_videos()

  if event.key is 'enter':

14:54:52.566:dazbo-yt-demos - INF: Downloads progress: 1/4
14:54:52.740:dazbo-yt-demos - INF: Getting: Sigrid - Burning Bridges
14:54:52.748:dazbo-yt-demos - INF: Downloading video Sigrid - Burning Bridges.mp4 ...


14:54:53.103:dazbo-yt-demos - INF: Creating audio...


MoviePy - Writing audio in /content/dazbo-yt-demos/output/pytubefix_with_moviepy/Sigrid - Burning Bridges.mp3


14:55:07.922:dazbo-yt-demos - INF: Done
14:55:07.923:dazbo-yt-demos - INF: Downloads progress: 2/4
14:55:08.055:dazbo-yt-demos - INF: Getting: Wolfenstein: The New Order - I Believe - Melissa Hollick (Official Ending Song)
14:55:08.068:dazbo-yt-demos - INF: Downloading video Wolfenstein_ The New Order - I Believe - Melissa Hollick _Official Ending Song_.mp4 ...


MoviePy - Done.


...

MoviePy - Done.


14:55:37.154:dazbo-yt-demos - INF: Creating audio...


MoviePy - Writing audio in /content/dazbo-yt-demos/output/pytubefix_with_moviepy/alyona alyona _ Jerry Heil - Teresa _ Maria _LIVE_ _ Ukraine __ _ Grand Final _ Eurovision 2024.mp3


14:55:44.463:dazbo-yt-demos - INF: Done
14:55:44.467:dazbo-yt-demos - INF: Downloads finished. See files in /content/dazbo-yt-demos/output/pytubefix_with_moviepy.


MoviePy - Done.


### Option 3 - With YT_DLP

I wanted to try the other popular YouTube package: [yt-dlp](https://pypi.org/project/yt-dlp/). The [repo](https://github.com/yt-dlp/yt-dlp) is a fork of the now unmaintained `youtube-dl`.

Pros:

- It is very powerful, with far more options and features than `pytubefix`.
- It can be installed as a standalone command-line executable, or as a pip-installable Python package.
- Sets mp3 headers properly!
- It has some powerful and network proxy settings. This can be useful if, for example, you are trying to download videos that are geo-restricted.

Cons:

- It is more complicated to use.
- The documentation is complex and somewhat hard to understand. And there's no real Python-specific documentation.
- It depends on having ffmpeg installed for some use cases.
- It is significantly slower that `pytubefix` for performing video download and audio extraction.


In [12]:
import yt_dlp

output_locn = f"{locations.output_dir}/yt_dlp"

def process_yt_videos():
    for i, url in enumerate(urls):
        logger.info(f"Downloads progress: {i+1}/{len(urls)}")

        try:
            # Options for downloading the video
            video_opts = {
                'format': 'best',  # Download the best quality video
                'outtmpl': f'{output_locn}/%(title)s.%(ext)s',  # Save video in output directory
            }

            # Download the video
            with yt_dlp.YoutubeDL(video_opts) as ydl:
                logger.info("Downloading video...")
                ydl.download([url])

            # Options for extracting audio and saving as MP3
            audio_opts = {
                'format': 'bestaudio',  # Download the best quality audio
                'outtmpl': f'{output_locn}/%(title)s.%(ext)s',  # Save audio in output directory
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3',
                }],
            }

            # Download and extract audio
            with yt_dlp.YoutubeDL(audio_opts) as ydl:
                logger.info("Extracting and saving audio as MP3...")
                ydl.download([url])

        except Exception as e:
            logger.error(f"Error processing URL '{url}'.")
            logger.debug(f"The cause was: {e}")

    logger.info(f"Downloads finished. Check out files at {output_locn}.")

process_yt_videos()

14:55:44.628:dazbo-yt-demos - INF: Downloads progress: 1/4
14:55:44.832:dazbo-yt-demos - INF: Downloading video...


[youtube] Extracting URL: https://www.youtube.com/watch?v=udRAIF6MOm8
[youtube] udRAIF6MOm8: Downloading webpage
[youtube] udRAIF6MOm8: Downloading ios player API JSON
[youtube] udRAIF6MOm8: Downloading mweb player API JSON
[youtube] udRAIF6MOm8: Downloading player 81ef9024
[youtube] udRAIF6MOm8: Downloading m3u8 information
[info] udRAIF6MOm8: Downloading 1 format(s): 18
[download] Destination: /content/dazbo-yt-demos/output/yt_dlp/Sigrid - Burning Bridges.mp4
[download] 100% of    4.78MiB in 00:00:00 at 15.37MiB/s  


14:55:50.419:dazbo-yt-demos - INF: Extracting and saving audio as MP3...


[youtube] Extracting URL: https://www.youtube.com/watch?v=udRAIF6MOm8
[youtube] udRAIF6MOm8: Downloading webpage
[youtube] udRAIF6MOm8: Downloading ios player API JSON
[youtube] udRAIF6MOm8: Downloading mweb player API JSON
[youtube] udRAIF6MOm8: Downloading m3u8 information
[info] udRAIF6MOm8: Downloading 1 format(s): 251
[download] Destination: /content/dazbo-yt-demos/output/yt_dlp/Sigrid - Burning Bridges.webm
[download] 100% of    2.76MiB in 00:00:00 at 13.95MiB/s  
[ExtractAudio] Destination: /content/dazbo-yt-demos/output/yt_dlp/Sigrid - Burning Bridges.mp3
Deleting original file /content/dazbo-yt-demos/output/yt_dlp/Sigrid - Burning Bridges.webm (pass -k to keep)


14:56:01.142:dazbo-yt-demos - INF: Downloads progress: 2/4
14:56:01.301:dazbo-yt-demos - INF: Downloading video...


...

14:56:28.804:dazbo-yt-demos - INF: Downloads progress: 4/4
14:56:28.940:dazbo-yt-demos - INF: Downloading video...


[youtube] Extracting URL: https://www.youtube.com/watch?v=d4N82wPpdg8
[youtube] d4N82wPpdg8: Downloading webpage
[youtube] d4N82wPpdg8: Downloading ios player API JSON
[youtube] d4N82wPpdg8: Downloading mweb player API JSON
[youtube] d4N82wPpdg8: Downloading m3u8 information
[info] d4N82wPpdg8: Downloading 1 format(s): 18
[download] Destination: /content/dazbo-yt-demos/output/yt_dlp/alyona alyona & Jerry Heil - Teresa & Maria (LIVE) ｜ Ukraine 🇺🇦 ｜ Grand Final ｜ Eurovision 2024.mp4
[download] 100% of   15.89MiB in 00:00:00 at 44.71MiB/s  


14:56:32.189:dazbo-yt-demos - INF: Extracting and saving audio as MP3...


[youtube] Extracting URL: https://www.youtube.com/watch?v=d4N82wPpdg8
[youtube] d4N82wPpdg8: Downloading webpage
[youtube] d4N82wPpdg8: Downloading ios player API JSON
[youtube] d4N82wPpdg8: Downloading mweb player API JSON
[youtube] d4N82wPpdg8: Downloading m3u8 information
[info] d4N82wPpdg8: Downloading 1 format(s): 251
[download] Destination: /content/dazbo-yt-demos/output/yt_dlp/alyona alyona & Jerry Heil - Teresa & Maria (LIVE) ｜ Ukraine 🇺🇦 ｜ Grand Final ｜ Eurovision 2024.webm
[download] 100% of    2.93MiB in 00:00:00 at 14.62MiB/s  
[ExtractAudio] Destination: /content/dazbo-yt-demos/output/yt_dlp/alyona alyona & Jerry Heil - Teresa & Maria (LIVE) ｜ Ukraine 🇺🇦 ｜ Grand Final ｜ Eurovision 2024.mp3
Deleting original file /content/dazbo-yt-demos/output/yt_dlp/alyona alyona & Jerry Heil - Teresa & Maria (LIVE) ｜ Ukraine 🇺🇦 ｜ Grand Final ｜ Eurovision 2024.webm (pass -k to keep)


14:56:44.976:dazbo-yt-demos - INF: Downloads finished. Check out files at /content/dazbo-yt-demos/output/yt_dlp.


### Conclusion

If you:

- Want to just download the videos and/or audio in the simplest and fastest way possible, then go with [Option 1](#option-1---with-pytubefix).
- Want to download the videos and/or audio and then carry out some sort of manipulation or conversion of the media, go with [Option 2](#option-2---pytubefix-and-moviepy).
- If you want out-of-the-box proxy configuration, e.g. to bypass geo-restrictions, then go with [Option 3](#option-3---with-yt_dlp).

## Transcribing Audio to Text

### Extracting Audio Using Python Speech Recognition

The Python `speech_recognition` package has a number of built in `Recognizer` implementations.Here I'm using the [Google Web Speech API](https://wicg.github.io/speech-api/) `Recognizer`, which has its default API key hard coded into the Python `speech_recognition` library. It is free, but has some limitations. For example, it only allows a max of 60s segments.

In [13]:
%pip install --upgrade --no-cache-dir pydub SpeechRecognition ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 240.3 MB/s eta 0:00:00


In [14]:
import speech_recognition as sr
from pydub import AudioSegment
import ffmpeg
import traceback

In [15]:
def divide_chunks(sound, segment_size_secs=60):
    """ Split audio file into 60s chunks """

    segment_size_ms = segment_size_secs*1000
    for start_idx in range(0, len(sound), segment_size_ms):
        # Yield a chunk of audio data from start_idx to start_idx + segment_size_ms
        yield sound[start_idx:start_idx + segment_size_ms]


def transcribe_audio():
    """ Use Speech Recognition API with Google Web Speech API
    to convert audio dialog to text """
    recogniser = sr.Recognizer()
    for mp3_file in Path(output_locn).glob(f'*.mp3'):
        transcribe_audio_file(recogniser, mp3_file)

def transcribe_audio_file(recogniser, mp3_file, language="en-US"):
    logger.info(f"Converting {mp3_file}...")
    try:
        audio = AudioSegment.from_file(mp3_file)
            # If AudioSegment is not working - e.g. due to broken mp3 headers - we
            # can use ffmpeg as a workaround. However, it's a lot slower.
            # ffmpeg.input(mp3_file).output(wav_file).run() # Convert with ffmpeg
            # logger.info(f"Successfully converted {mp3_file} to {wav_file}.")
            # audio = AudioSegment.from_wav(wav_file) # Read the audio

        segments = list(divide_chunks(audio, segment_size_secs=60)) # split the wav into 60s segments
        transcription_extracts = {}
        for index, chunk in enumerate(segments):
            with io.BytesIO() as wav_io:
                chunk.export(wav_io, format='wav')
                wav_io.seek(0)  # Move to the start of the BytesIO object before reading from it

                with sr.AudioFile(wav_io) as source:
                    audio_data = recogniser.record(source)

                try:
                    extracted = recogniser.recognize_google(audio_data, language=language)
                    logger.debug(f"Chunk {index} extracted.")
                    transcription_extracts[index] = extracted
                except sr.UnknownValueError:
                        # Log the unknown value error and continue
                    logger.warning(f"Chunk {index}: Could not understand the audio. Maybe it was empty.")

        logger.info("Extract:")
        for idx, extract in transcription_extracts.items():
            logger.info(f"{idx}: {extract}")

    except ffmpeg.Error as e:
        logger.error(f"FFmpeg failed to convert {mp3_file}: {str(e)}")
    except Exception as e:
        logger.error("Unexpected error.", exc_info=True)

transcribe_audio()
logger.info("Done")

14:56:49.295:dazbo-yt-demos - INF: Converting /content/dazbo-yt-demos/output/yt_dlp/Wolfenstein： The New Order - I Believe - Melissa Hollick (Official Ending Song).mp3...
14:56:57.201:dazbo-yt-demos - WRN: Chunk 0: Could not understand the audio. Maybe it was empty.
14:57:12.620:dazbo-yt-demos - DBG: Chunk 1 extracted.
14:57:19.115:dazbo-yt-demos - WRN: Chunk 2: Could not understand the audio. Maybe it was empty.
14:57:27.912:dazbo-yt-demos - WRN: Chunk 3: Could not understand the audio. Maybe it was empty.
14:57:39.230:dazbo-yt-demos - DBG: Chunk 4 extracted.
14:57:43.609:dazbo-yt-demos - WRN: Chunk 5: Could not understand the audio. Maybe it was empty.
14:57:43.612:dazbo-yt-demos - INF: Extract:
14:57:43.613:dazbo-yt-demos - INF: 1: I believe that the stars keep shining us through the night I believe if we just keep trying it'll be alright I believe it's Sunday we're going to find a way and I believe in a beautiful day I believe in love is walking side by side I believe that someday 

### Results

- It fails to transcribe the Ukrainian song. Not too surprising, since this API does not detect language automatically, and defaults to recognising English.
- It does an amazing job with the Jim Carey speech.
- It is partially successful when transcribing songs.

### Conclusions

Well, not bad, but also, not great!  It's pretty good if there's no background sound or ambient noise.  But it's pretty poor when working with songs.

### Transcribing Ukrainian

Let's try and transcribe from the Ukrainian song:

In [16]:
def transcribe_ua():
    recogniser = sr.Recognizer()
    for mp3_file in Path(output_locn).glob(f'alyona*.mp3'):
        transcribe_audio_file(recogniser, mp3_file, language="uk-UA")

transcribe_ua()

14:58:51.293:dazbo-yt-demos - INF: Converting /content/dazbo-yt-demos/output/yt_dlp/alyona alyona & Jerry Heil - Teresa & Maria (LIVE) ｜ Ukraine 🇺🇦 ｜ Grand Final ｜ Eurovision 2024.mp3...
14:59:10.977:dazbo-yt-demos - DBG: Chunk 0 extracted.
14:59:24.425:dazbo-yt-demos - WRN: Chunk 1: Could not understand the audio. Maybe it was empty.
14:59:33.466:dazbo-yt-demos - WRN: Chunk 2: Could not understand the audio. Maybe it was empty.
14:59:36.385:dazbo-yt-demos - WRN: Chunk 3: Could not understand the audio. Maybe it was empty.
14:59:36.387:dazbo-yt-demos - INF: Extract:
14:59:36.388:dazbo-yt-demos - INF: 0: через джерело пропила собі шлях щоб щоб не було світла й плеча мені Money Money всі звивисті скелясті Але знай твоїй руці твоя власниця давай


### Results

Partial success.  But overall... Not great!

## Extract Existing Transcripts from Videos

Now I'm going to use the [youtube-transcript-api](https://github.com/jdepoix/youtube-transcript-api) to extract existing transcripts from YouTube videos. Not only will it return the transcript, but it can also be used to translate those to translate those transcripts into other languages.  So now I can download my Ukrainian song, and see both the Ukrainian transcript and the English translation. This is pretty awesome!

However, some videos do not contain transcripts.

In [17]:
%pip install --upgrade --no-cache-dir youtube_transcript_api

In [18]:
import youtube_transcript_api as yt_api
from pytubefix import YouTube
from pytubefix.cli import on_progress

def get_transcripts():
    """ Extract existing transcript data from videos """
    for url in urls:
        try: # Just so we can get the video title
            yt = YouTube(url, on_progress_callback=on_progress)
        except Exception as e:
            logger.error(f"Error processing URL '{url}'.")
            logger.debug(f"The cause was: {e}")
            continue

        logger.info(f"Processing '{yt.title}'...")
        video_id = get_video_id(url)

        try:
            # By default, we get a list of 1: only get the preferred language transcript
            transcript_list = yt_api.YouTubeTranscriptApi.list_transcripts(video_id)
        except Exception as e:
            logger.error(f"Unable to extract transcript for '{yt.title}'.")
            logger.debug(e)
            continue

        # iterate over all available transcripts
        for transcript in transcript_list:
            # The Transcript object provides metadata properties. Here are some...
            properties = {
                "video_id": transcript.video_id,
                "language": transcript.language,
                "language_code": transcript.language_code,
                "is_generated": transcript.is_generated,  # Whether it has been manually created or generated by YouTube
                "is_translatable": transcript.is_translatable,  # Whether this transcript can be translated or not
                "translation_languages": transcript.translation_languages,
            }

            for prop, value in properties.items():
                logger.info(f"{prop}: {value}")

            # Fetch the actual transcript data
            transcript_data = transcript.fetch() # returns a list of dicts
            logger.info(f"Raw transcript:\n{transcript_data}")

            processed_transcript = process_transcript(transcript_data)
            logger.info(f"Processed transcript:\n{processed_transcript}")

            # Translate to en if we can
            if (transcript.language_code != "en" and
                    transcript.is_translatable and
                    any(lang['language_code'] == 'en' for lang in transcript.translation_languages)):
                transcript_data = transcript.translate('en').fetch() # translate to en
                processed_transcript = process_transcript(transcript_data)
                logger.info(f"Processed translated transcript:\n{processed_transcript}")

def process_transcript(transcript_data):
    """ Get all entries that are of type 'text' and NOT starting with [ """
    return "\n".join([entry['text'] for entry in transcript_data
                                     if entry['text'][0] != "["])

get_transcripts()

14:59:39.649:dazbo-yt-demos - INF: Processing 'Sigrid - Burning Bridges'...
14:59:40.393:dazbo-yt-demos - ERR: Unable to extract transcript for 'Sigrid - Burning Bridges'.
14:59:40.395:dazbo-yt-demos - DBG: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=udRAIF6MOm8! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
14:59:40.471:dazbo-yt-demos - INF: Processing 'Wolfenstein: The New Order - I Believe - Melissa Hollick (Official Ending Song)'...
14:59:41.018:dazbo-yt-demos - INF: video_id: CiTn4j7gVvY
14:59:41.020:dazbo-yt-demos - IN

How cool is this!?

## Adding Google Cloud Smarts

Let's integrate some Google Cloud Vertex AI smarts. Start by installing the **Google Cloud Vertex AI SDK for Python**.

From [Introduction to the Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/docs/python-sdk/use-vertex-ai-python-sdk#sdk-vs-client-library):

When you install the Vertex AI SDK for Python (`google.cloud.aiplatform`), the Vertex AI Python client library (`google.cloud.aiplatform.gapic`) is also installed. The Vertex AI SDK and the Vertex AI Python client library provide similar functionality with different levels of granularity. The Vertex AI SDK operates at a higher level of abstraction than the client library and is suitable for most common data science workflows. If you need lower-level functionality, then use the Vertex AI Python client library.

In [19]:
# Install Vertex AI SDK for Python and Vertex Generative AI SDK for Python
%pip install --upgrade google-cloud-aiplatform google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 30.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.70.0
    Uninstalling google-cloud-aiplatform-1.70.0:
      Successfully uninstalled google-cloud-aiplatform-1.70.0


In [20]:
from google.cloud import aiplatform # Google Cloud Vertex AI SDK for Python
# import vertexai   # Google Cloud Vertex Generative AI SDK for Python
import google.generativeai as genai  # Google Gemini API (GenAI)
from vertexai.generative_models import GenerativeModel

import sys
from getpass import getpass

# If we're running Google Colab, authenticate
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

# Retrieve PROJECT_ID and other variables from any .env we can find
try:
    dc.get_envs_from_file()
except ValueError as e:
    logger.error(f"Problem reading env file:\n{e}")

env_vars = ["PROJECT_ID", "REGION"] # The vars we want to retrieve
for env_var in env_vars:
    if not os.getenv(env_var):
        gcp_project_id = '' # @param {type: "string"}
        # If not retrieved from .env we'll need to input the value
        os.environ[env_var] = getpass(f"Enter {env_var}: ")

    # Set Python variable of the same name as the env var, e.g. PROJECT_ID
    globals()[env_var] = os.environ[env_var]
    val = globals()[env_var]
    if env_var == "PROJECT_ID":
        logger.info(f"{env_var} retrieved: {val[-3:]}")
    else:
        logger.info(f"{env_var} retrieved: {val}")

DEBUG:__name__:Trying .env at /content/.env
DEBUG:__name__:Trying .env at /.env
DEBUG:__name__:Trying .env at /.env
15:05:40.924:dazbo-yt-demos - INF: PROJECT_ID retrieved: 345


Enter REGION: ··········


15:17:34.574:dazbo-yt-demos - INF: REGION retrieved: FR


Only run the next cell if you want to manually clear the environment variables and then input new values. In this scenario, you'll also want to comment out any variables in your .env file.

In [21]:
# Optionally run this if we want to clear env vars
for env_var in env_vars:
    if env_var in os.environ:
        del os.environ[env_var]
        logger.info(f"Cleared environment variable: {env_var}")

15:17:34.598:dazbo-yt-demos - INF: Cleared environment variable: PROJECT_ID
15:17:34.600:dazbo-yt-demos - INF: Cleared environment variable: REGION
